In [1]:
# import torch
# print(torch.cuda.memory_allocated())
# print(torch.cuda.memory_reserved())

# torch.cuda.empty_cache()   
# print(torch.cuda.memory_allocated())
# print(torch.cuda.memory_reserved())
# print(torch.cuda.max_memory_allocated(device=None))

In [2]:
#!pip install pynvml

In [3]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved

In [4]:
#!nvidia-smi

In [5]:
#! pip install datasets transformers[sentencepiece]

In [6]:
import datasets

snli = datasets.load_dataset('snli', split='train')
mnli = datasets.load_dataset('glue', 'mnli', split='train')


Reusing dataset snli (C:\Users\basuc\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)
Reusing dataset glue (C:\Users\basuc\.cache\huggingface\datasets\glue\mnli\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [7]:
snli.column_names

['premise', 'hypothesis', 'label']

In [8]:
mnli.column_names

['premise', 'hypothesis', 'label', 'idx']

In [9]:
mnli[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [10]:
mnli = mnli.remove_columns("idx")

In [11]:
#del dataset

In [12]:
snli = snli.cast(mnli.features)

dataset = datasets.concatenate_datasets([snli, mnli])

del snli, mnli

Loading cached processed dataset at C:\Users\basuc\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-77b5f68f4313ad61.arrow


In [13]:
print(f"before: {len(dataset)} rows")
dataset = dataset.filter(
    lambda x: True if x['label'] == 0 else False ## only entailment remains
)
print(f"after: {len(dataset)} rows")

Loading cached processed dataset at C:\Users\basuc\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-afdbe693143c1078.arrow


before: 942854 rows
after: 314315 rows


In [14]:
from transformers import BertTokenizer
#from transformers import DistilBertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

dataset = dataset.map(
    lambda x: tokenizer(
            x['premise'], max_length=128, padding='max_length',
            truncation=True
        ), batched=True
)

dataset = dataset.rename_column('input_ids', 'anchor_ids')
dataset = dataset.rename_column('attention_mask', 'anchor_mask')

dataset

Loading cached processed dataset at C:\Users\basuc\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-2bff538a3d03b3cd.arrow


Dataset({
    features: ['anchor_mask', 'hypothesis', 'anchor_ids', 'label', 'premise', 'token_type_ids'],
    num_rows: 314315
})

In [15]:
dataset = dataset.map(
    lambda x: tokenizer(
            x['hypothesis'], max_length=128, padding='max_length',
            truncation=True
    ), batched=True
)

dataset = dataset.rename_column('input_ids', 'positive_ids')
dataset = dataset.rename_column('attention_mask', 'positive_mask')



Loading cached processed dataset at C:\Users\basuc\.cache\huggingface\datasets\snli\plain_text\1.0.0\1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b\cache-4939be4049a780d2.arrow


In [16]:
#dataset = dataset.remove_columns(['premise', 'hypothesis', 'label']) #, 'token_type_ids'
dataset = dataset.remove_columns(['premise', 'hypothesis', 'label','token_type_ids']) #, 'token_type_ids'
dataset

Dataset({
    features: ['anchor_ids', 'anchor_mask', 'positive_mask', 'positive_ids'],
    num_rows: 314315
})

In [17]:
dataset.set_format(type='torch', output_all_columns=True)

In [18]:
len(dataset['anchor_ids'])

314315

In [19]:
import torch

batch_size = 32

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [20]:
from transformers import BertModel
#from transformers import DistilBertModel

# start from a pretrained bert-base-uncased model
model = BertModel.from_pretrained('bert-base-uncased')
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# define mean pooling function
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

In [22]:
# set device and move model there
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
##device= torch.device('cpu')
model.to(device)
print(f'moved to {device}')

moved to cuda


In [23]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

439069696
494927872


In [24]:
# define layers to be used in multiple-negatives-ranking
cos_sim = torch.nn.CosineSimilarity()
loss_func = torch.nn.CrossEntropyLoss()
scale = 20.0  # we multiply similarity score by this scale value
# move layers to device
cos_sim.to(device)
loss_func.to(device)

CrossEntropyLoss()

In [25]:
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AdamW

# initialize Adam optimizer
#optim = torch.optim.Adam(model.parameters(), lr=2e-5)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.1},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optim = AdamW(optimizer_grouped_parameters, lr=2e-5)

# setup warmup for first ~10% of steps
total_steps = int(len(dataset['anchor_ids']) / batch_size)
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optim, num_warmup_steps=warmup_steps,
    num_training_steps=total_steps-warmup_steps
)

In [26]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [27]:
# !ls /content/gdrive/MyDrive/BERTfinetune

In [28]:
# model_path = F"/content/gdrive/MyDrive/BERTfinetune" 

In [29]:
from tqdm.auto import tqdm

epochs = 1
# 1 epoch should be enough, increase if wanted
for epoch in range(epochs):
    model.train()  # make sure model is in training mode
    # initialize the dataloader loop with tqdm (tqdm == progress bar)
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # zero all gradients on each new step
        optim.zero_grad()
        # prepare batches and more all to the active device
        anchor_ids = batch['anchor_ids'].to(device)
        anchor_mask = batch['anchor_mask'].to(device)
        pos_ids = batch['positive_ids'].to(device)
        pos_mask = batch['positive_mask'].to(device)
        # extract token embeddings from BERT
        a = model(
            anchor_ids, attention_mask=anchor_mask
        )[0]  # all token embeddings
        p = model(
            pos_ids, attention_mask=pos_mask
        )[0]
        # get the mean pooled vectors
        a = mean_pool(a, anchor_mask)
        p = mean_pool(p, pos_mask)
        # calculate the cosine similarities
        scores = torch.stack([
            cos_sim(
                a_i.reshape(1, a_i.shape[0]), p
            ) for a_i in a])
        # get label(s) - we could define this before if confident of consistent batch sizes
        labels = torch.tensor(range(len(scores)), dtype=torch.long, device=scores.device)
        # and now calculate the loss
        loss = loss_func(scores*scale, labels)
        # using loss, calculate gradients and then optimize
        loss.backward()
        optim.step()
        # update learning rate scheduler
        scheduler.step()
        # update the TDQM progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 6.00 GiB total capacity; 4.43 GiB already allocated; 40.81 MiB free; 4.48 GiB reserved in total by PyTorch)

In [ ]:
import os

model_path = './bert_uncased_test_mnr'

if not os.path.exists(model_path):
    os.mkdir(model_path)

model.save_pretrained(model_path)